In [40]:
import pandas as pd

# load data
data = pd.read_csv("./fnb_datav2.csv")

data = data.drop(columns=["item_descrip"])
data.head()

,idcol,interaction,int_date,item,page,tod,item_type,segment,beh_segment,active_ind
0,755,DISPLAY,17JAN2023,NONE,Screen1,Afternoon,ALL,segment3,B01,Semi Active
1,4521,DISPLAY,27FEB2023,NONE,Screen1,Afternoon,ALL,segment1,B07,Semi Active
2,4521,DISPLAY,18FEB2023,NONE,Screen1,Afternoon,ALL,segment1,B07,Semi Active
3,4521,DISPLAY,30JAN2023,NONE,Screen1,Morning,ALL,segment1,B07,Semi Active
4,4521,CLICK,05FEB2023,IBAB,Screen1,Afternoon,INSURE,segment1,B07,Semi Active


## Creating a interaction matgrix

In [41]:
# Pivot the DataFrame to create a user-item interaction matrix
#For simplicity, we'll use 'interaction' as the value to fill, you can change this as needed
interaction_matrix = data.pivot_table(index='idcol', columns='item', values='interaction', aggfunc='count', fill_value=0)
interaction_matrix

item,CABC,CACU,CAFB,CAFI,CAFM,CAFS,CAFU,CALI,CANL,CAPO,...,MMMC,MMSM,NACS,NAFW,NASD,NATR,NONE,SEVP,WHCR,XCFL
idcol,,,,,,,,,,,,,,,,,,,,,
755,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
4521,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,3,0,0,0
6145,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,4,0,0,0
7125,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
8469,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,3,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25685663618,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,2,0,0,1
25693117918,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
25705258018,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0


# EDA

In [42]:
data.nunique()

idcol          84375
interaction        3
int_date          88
item             104
page               2
tod                4
item_type          7
segment            4
beh_segment       50
active_ind         3
dtype: int64

## working out weights

### Normalize according to number of interactions/user


In [43]:
interaction_matrix = data.pivot_table(index='idcol', columns='item', values='interaction', aggfunc='count', fill_value=0)
# Normalize each row by the total count of values in the row
normalized_matrix = interaction_matrix.div(interaction_matrix.sum(axis=1), axis=0)

# Display the normalized matrix
display(normalized_matrix)

# Calculate and display the sum of each row in the normalized matrix
row_sums = normalized_matrix.sum(axis=1)
print("\nSum of each row (should be 1.0):")
print(row_sums)

item,CABC,CACU,CAFB,CAFI,CAFM,CAFS,CAFU,CALI,CANL,CAPO,...,MMMC,MMSM,NACS,NAFW,NASD,NATR,NONE,SEVP,WHCR,XCFL
idcol,,,,,,,,,,,,,,,,,,,,,
755,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.0,1.000000,0.0,0.0,0.000000
4521,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.600000,0.0,0.0,0.000000
6145,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.0,1.000000,0.0,0.0,0.000000
7125,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.0,1.000000,0.0,0.0,0.000000
8469,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.0,1.000000,0.0,0.0,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25685663618,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.666667,0.0,0.0,0.333333
25693117918,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.200000,0.0,0.0,0.000000
25705258018,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.142857,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000



Sum of each row (should be 1.0):
idcol
755            1.0
4521           1.0
6145           1.0
7125           1.0
8469           1.0
              ... 
25685663618    1.0
25693117918    1.0
25705258018    1.0
25721743718    1.0
25728632418    1.0
Length: 84375, dtype: float64


### weight based on interaction types 
[display - 0   click - 1   checkout - 2]

In [44]:
# Define weights for different types of interactions
weight_dict = {'DISPLAY': 0, 'CLICK': 1, 'CHECKOUT': 3}
# Add a new column for weighted count of interactions
data['weighted_interaction'] = data['interaction'].map(weight_dict)
# Create the interaction matrix with weighted counts
interaction_matrix_weighted = data.pivot_table(index='idcol', columns='item', values='weighted_interaction', aggfunc='sum', fill_value=0)

#interaction_matrix.div(interaction_matrix.sum(axis=1), axis=0)
# Normalize each row by the total count of values in the row             ADD OR REMOVE WEIGHTED?
normalized_matrix_weighted = interaction_matrix_weighted.div(interaction_matrix_weighted.sum(axis=1), axis=0)

# Display the normalized matrix
print("Normalized Interaction Matrix with Weighted Interactions:")
print(normalized_matrix_weighted)

Normalized Interaction Matrix with Weighted Interactions:
item         CABC  CACU  CAFB  CAFI  CAFM  CAFS  CAFU  CALI  CANL  CAPO  ...  \
idcol                                                                    ...   
755           NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN  ...   
4521          0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0  ...   
6145          NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN  ...   
7125          NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN  ...   
8469          NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN  ...   
...           ...   ...   ...   ...   ...   ...   ...   ...   ...   ...  ...   
25685663618   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0  ...   
25693117918   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0  ...   
25705258018   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0  ...   
25721743718   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   

In [45]:
# Normalize each row by the total count of values in the row
row_sums = interaction_matrix_weighted.sum(axis=1)
# Avoid division by zero by setting zero row sums to 1 temporarily
row_sums = row_sums.replace(0, 1)
normalized_matrix_weighted = interaction_matrix.div(row_sums, axis=0)

# Ensure that rows which originally had sum 0 remain 0 after normalization
normalized_matrix_weighted = normalized_matrix_weighted.replace(1.0, 0.0)

# Display the normalized matrix
print("Normalized Interaction Matrix with Weighted Interactions:")
print(normalized_matrix_weighted)

Normalized Interaction Matrix with Weighted Interactions:
item         CABC  CACU  CAFB  CAFI  CAFM  CAFS  CAFU  CALI  CANL  CAPO  ...  \
idcol                                                                    ...   
755           0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0  ...   
4521          0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0  ...   
6145          0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0  ...   
7125          0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0  ...   
8469          0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0  ...   
...           ...   ...   ...   ...   ...   ...   ...   ...   ...   ...  ...   
25685663618   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0  ...   
25693117918   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0  ...   
25705258018   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0  ...   
25721743718   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   

# Trying Algos

In [57]:
from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors
from sklearn.model_selection import train_test_split

# train_data, test_data = train_test_split(data, test_size=0.1, random_state=42)
interaction_matrix = data.pivot_table(index='idcol', columns='item', values='interaction', aggfunc='count', fill_value=0).T
print(interaction_matrix.shape)
display(interaction_matrix)

# train_data = train_data.T
# test_data = test_data.T

matrix = csr_matrix(interaction_matrix)

knn = NearestNeighbors(metric='cosine', algorithm='brute')
knn.fit(matrix)



(104, 84375)


idcol,755,4521,6145,7125,8469,13768,14454,15000,15784,22210,...,25677977618,25678849118,25680243518,25681314518,25685525018,25685663618,25693117918,25705258018,25721743718,25728632418
item,,,,,,,,,,,,,,,,,,,,,
CABC,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
CACU,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
CAFB,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
CAFI,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
CAFM,0,0,0,0,0,0,2,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
NATR,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
NONE,1,3,4,1,3,1,1,1,1,2,...,2,1,1,1,3,2,1,0,1,5
SEVP,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


NearestNeighbors(algorithm='brute', metric='cosine')

In [113]:
def fitknn(matrix,metric):
    matrix = csr_matrix(matrix)
    knn = NearestNeighbors(metric=metric, algorithm='brute')
    knn.fit(matrix)
    return knn

In [67]:
def print_recs(interaction_matrix,distances,indices):
    for i in range(len(distances.flatten())):
        if i == 0:
            print('Recommendations for {0}:\n'.format(interaction_matrix.index[3]))
            
        else:
            print('{0}: {1}, with distance of {2}:'.format(i, interaction_matrix.index[indices.flatten()[i]], distances.flatten()[i]))
            # break

In [97]:
interaction_matrix = data.pivot_table(index='idcol', columns='item', values='interaction', aggfunc='count', fill_value=0).T
fitknn(interaction_matrix,'cosine')
distances, indices = knn.kneighbors(interaction_matrix.iloc[3,:].values.reshape(1, -1), n_neighbors = 10)
print(distances)
print(indices)
print_recs(interaction_matrix,distances,indices)
print(len(distances.flatten()))
# print(test_data.index[indices.flatten()[0]])
print(indices.flatten()[0])

[[7.77156117e-16 9.01445266e-01 9.10965272e-01 9.11532129e-01
  9.15376451e-01 9.20569049e-01 9.27284282e-01 9.35311039e-01
  9.48825207e-01 9.49717006e-01]]
[[  3  28  24  11  10  12  33 100  31  47]]
Recommendations for CAFI:

1: CCLI, with distance of 0.9014452662221031:
2: CCAI, with distance of 0.9109652715872409:
3: CARF, with distance of 0.9115321291138396:
4: CARE, with distance of 0.9153764510715952:
5: CASD, with distance of 0.9205690487730216:
6: CUPL, with distance of 0.9272842822019907:
7: NONE, with distance of 0.9353110388123935:
8: CTLN, with distance of 0.9488252069128648:
9: EBKA, with distance of 0.9497170063193094:
10
3


### KNN for normalized matrix

In [98]:
normalized_matrix = interaction_matrix.div(interaction_matrix.sum(axis=0), axis=1)
#display(normalized_matrix)
# col_sums = normalized_matrix.sum(axis=0)
# print("\nSum of each column (should be 1.0):")
# print(col_sums)
fitknn(normalized_matrix,'cosine')
#normalized_matrix = normalized_matrix.T
distances, indices = knn.kneighbors(normalized_matrix.iloc[3,:].values.reshape(1, -1), n_neighbors = 10)
# print(distances)
# print(indices)
print_recs(normalized_matrix,distances,indices)

Recommendations for CAFI:

1: CCLI, with distance of 0.9420329001849983:
2: CCAI, with distance of 0.9480590067640622:
3: NONE, with distance of 0.9526488380225258:
4: CARE, with distance of 0.9529361458438198:
5: CARF, with distance of 0.9550353077993446:
6: CUPL, with distance of 0.9675396800297845:
7: CASD, with distance of 0.9677776862957561:
8: EBWP, with distance of 0.97457803311461:
9: EBEM, with distance of 0.9755482277265526:


### KNN for weighted interaction type

In [101]:
# load data
data = pd.read_csv("./fnb_datav2.csv")
# Define weights for different types of interactions
weight_dict = {'DISPLAY': 0, 'CLICK': 1, 'CHECKOUT': 2}
# Add a new column for weighted count of interactions
data['weighted_interaction'] = data['interaction'].map(weight_dict)
# Create the interaction matrix with weighted counts
interaction_matrix_weighted = data.pivot_table(index='idcol', columns='item', values='weighted_interaction', aggfunc='sum', fill_value=0)
interaction_matrix_weighted = interaction_matrix_weighted.T
#print(interaction_matrix_weighted)
fitknn(interaction_matrix_weighted,'cosine')
distances, indices = knn.kneighbors(interaction_matrix_weighted.iloc[3,:].values.reshape(1, -1), n_neighbors = 10)
# print(distances)
# print(indices)
print_recs(interaction_matrix_weighted,distances,indices)


Recommendations for CAFI:

1: CCLI, with distance of 0.9032216250458793:
2: CCAI, with distance of 0.9133331578295651:
3: CARE, with distance of 0.9157081194698464:
4: CARF, with distance of 0.9164477265943239:
5: CASD, with distance of 0.921488864573397:
6: CUPL, with distance of 0.927412905410064:
7: NONE, with distance of 0.9349632553983405:
8: CTLN, with distance of 0.9460207592753302:
9: EBKA, with distance of 0.9492074028179507:


### KNN for normalized and weighted interaction type

In [103]:
#interaction_matrix.div(interaction_matrix.sum(axis=1), axis=0)
# Normalize each row by the total count of values in the row             ADD OR REMOVE WEIGHTED?
normalized_matrix_weighted = interaction_matrix_weighted.div(interaction_matrix.sum(axis=1), axis=0)
fitknn(normalized_matrix_weighted,'cosine')
# Display the normalized matrix
print("Normalized Interaction Matrix with Weighted Interactions:")
#print(normalized_matrix_weighted)
distances, indices = knn.kneighbors(normalized_matrix_weighted.iloc[3,:].values.reshape(1, -1), n_neighbors = 10)
# print(distances)
# print(indices)
print_recs(normalized_matrix_weighted,distances,indices)

Normalized Interaction Matrix with Weighted Interactions:
Recommendations for CAFI:

1: CCLI, with distance of 0.9032216250458793:
2: CCAI, with distance of 0.9133331578295651:
3: CARE, with distance of 0.9157081194698464:
4: CARF, with distance of 0.9164477265943239:
5: CASD, with distance of 0.921488864573397:
6: CUPL, with distance of 0.927412905410064:
7: NONE, with distance of 0.9349632553983405:
8: CTLN, with distance of 0.9460207592753302:
9: EBKA, with distance of 0.9492074028179507:


### KNN Comparison

In [114]:
matrices = [interaction_matrix,normalized_matrix,interaction_matrix_weighted,normalized_matrix_weighted]
names = ['interaction', 'normalized', 'weighted','normalized weighted']
metric = 'cosine'
for matrix,name in zip(matrices, names):
    knn = fitknn(matrix,metric)
    distances, indices = knn.kneighbors(matrix.iloc[3,:].values.reshape(1, -1), n_neighbors = 5)
    print(f'{name}:')
    print_recs(matrix,distances,indices)


interaction:
Recommendations for CAFI:

1: CCLI, with distance of 0.9014452662221031:
2: CCAI, with distance of 0.9109652715872409:
3: CARF, with distance of 0.9115321291138396:
4: CARE, with distance of 0.9153764510715952:
normalized:
Recommendations for CAFI:

1: CCLI, with distance of 0.9575365837418601:
2: CARE, with distance of 0.9649913389201571:
3: CCAI, with distance of 0.9706886876980377:
4: CARF, with distance of 0.9731854915890479:
weighted:
Recommendations for CAFI:

1: CCLI, with distance of 0.9020473728681493:
2: CCAI, with distance of 0.9143505596876553:
3: CARE, with distance of 0.9157397863586183:
4: CARF, with distance of 0.9171692777124184:
normalized weighted:
Recommendations for CAFI:

1: CCLI, with distance of 0.9020473728681488:
2: CCAI, with distance of 0.9143505596876551:
3: CARE, with distance of 0.9157397863586184:
4: CARF, with distance of 0.9171692777124185:


In [115]:
matrices = [interaction_matrix,normalized_matrix,interaction_matrix_weighted,normalized_matrix_weighted]
names = ['interaction', 'normalized', 'weighted','normalized weighted']
metric = 'euclidean'
for matrix,name in zip(matrices, names):
    knn = fitknn(matrix,metric)
    distances, indices = knn.kneighbors(matrix.iloc[3,:].values.reshape(1, -1), n_neighbors = 5)
    print(f'{name}:')
    print_recs(matrix,distances,indices)

interaction:
Recommendations for CAFI:

1: IPSD, with distance of 77.75602870517501:
2: IPMX, with distance of 77.86526825228306:
3: IPFN, with distance of 77.92945527847606:
4: WHCR, with distance of 78.40280607223188:
normalized:
Recommendations for CAFI:

1: IPSD, with distance of 8.37844089202277:
2: IPST, with distance of 8.44291375676648:
3: GAFC, with distance of 8.447303497216854:
4: IPMX, with distance of 8.454367352834561:
weighted:
Recommendations for CAFI:

1: NONE, with distance of 107.22872749408155:
2: IPSD, with distance of 107.3126274023705:
3: IPMX, with distance of 107.49883720301351:
4: IPFN, with distance of 107.61970079869205:
normalized weighted:
Recommendations for CAFI:

1: NONE, with distance of 0.07425812153329749:
2: IBAA, with distance of 0.07832068732663507:
3: FIHC, with distance of 0.0788667400315092:
4: IBAB, with distance of 0.07969709223443149:


In [ ]:
matrices = [interaction_matrix,normalized_matrix,interaction_matrix_weighted,normalized_matrix_weighted]
names = ['interaction', 'normalized', 'weighted','normalized weighted']
metric = 'euclidean'
for matrix,name in zip(matrices, names):
    knn = fitknn(matrix,metric)
    distances, indices = knn.kneighbors(matrix.iloc[3,:].values.reshape(1, -1), n_neighbors = 5)
    print(f'{name}:')
    print_recs(matrix,distances,indices)